# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Why does my heatmap not showing correctly?
# Why does marker not working correctly?

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
weather_data = pd.read_csv("../WeatherPy/weather_data.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,East London,90,ZA,1579453770,88,-33.02,27.91,64.40,6.93
1,George Town,20,MY,1579453844,100,5.41,100.34,80.60,3.53
2,Busselton,0,AU,1579453829,42,-33.65,115.33,66.00,13.13
3,Prince Rupert,40,CA,1579454017,80,54.32,-130.32,39.20,11.41
4,Ribeira Grande,75,PT,1579454018,82,38.52,-28.70,66.20,16.11
...,...,...,...,...,...,...,...,...,...
564,Port-Gentil,20,GA,1579454092,74,-0.72,8.78,86.00,9.17
565,Cockburn Town,0,TC,1579454092,73,21.46,-71.14,75.31,20.87
566,Imbituba,40,BR,1579454092,58,-28.24,-48.67,84.20,11.41
567,Sawakin,94,SD,1579454092,76,19.11,37.33,73.06,18.32


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [28]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout = figure_layout)

In [33]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,  dissipating=False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness less than 20 and max temp between 50 and 90
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] <= 20) & \
                                   (weather_data["Max Temp"] > 50) & (weather_data["Max Temp"] < 90)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
1,George Town,20,MY,1579453844,100,5.41,100.34,80.60,3.53
5,Porto-Novo,20,BJ,1579454018,70,6.50,2.60,87.80,8.05
7,Esperance,3,AU,1579454018,76,-33.87,121.90,57.20,6.93
14,Kidal,0,ML,1579454019,19,18.44,1.41,68.61,8.68
15,Sao Filipe,5,CV,1579454019,74,14.90,-24.50,71.85,7.96
37,Danane,9,CI,1579454022,21,7.26,-8.15,84.76,2.77
40,Kirakira,13,SB,1579454022,83,-10.45,161.92,75.74,4.21
54,Buala,16,SB,1579454024,84,-8.14,159.59,78.71,7.45
60,Ahipara,0,NZ,1579454025,100,-35.17,173.17,60.01,3.00
67,Honiara,20,SB,1579454026,94,-9.43,159.95,75.20,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = narrowed_city_df[["City", "Lat", "Lng"]].reset_index()
hotel_df["Hotel Name"] = ""

hotel_df

,index,City,Lat,Lng,Hotel Name
0,7,Esperance,-33.87,121.90,
1,14,Kidal,18.44,1.41,
2,15,Sao Filipe,14.90,-24.50,
3,37,Danane,7.26,-8.15,
4,40,Kirakira,-10.45,161.92,
5,54,Buala,-8.14,159.59,
6,60,Ahipara,-35.17,173.17,
7,73,Cabo San Lucas,22.89,-109.91,
8,103,Tizimin,21.15,-88.15,
9,144,Araouane,18.90,-3.53,


In [21]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [23]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
print("-------End of Seach-------")

Retrieving Results for Index 0: Esperance.
Closest hotel in Esperance is Esperance YHA.
------------
Retrieving Results for Index 1: Kidal.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: Sao Filipe.
Closest hotel in Sao Filipe is Hotel Xaguate.
------------
Retrieving Results for Index 3: Danane.
Closest hotel in Danane is Hôtel ZAGOHI.
------------
Retrieving Results for Index 4: Kirakira.
Missing field/result... skipping.
------------
Retrieving Results for Index 5: Buala.
Closest hotel in Buala is Maringe Lagoon Lodge.
------------
Retrieving Results for Index 6: Ahipara.
Closest hotel in Ahipara is The Huts.
------------
Retrieving Results for Index 7: Cabo San Lucas.
Closest hotel in Cabo San Lucas is Villa Del Palmar Beach Resort & Spa Cabo San Lucas.
------------
Retrieving Results for Index 8: Tizimin.
Closest hotel in Tizimin is Hotel Roazi.
------------
Retrieving Results for Index 9: Araouane.
Missing field/result... skipping.
------------
Ret

In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [34]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))